In [1]:
import requests
import pandas as pd
import json
from cred import credentials
from request_api import get_genesis_api
from sql import get_data

In [62]:
data = pd.read_csv('all_merged_scaled_ur.csv', index_col=0)
data

,state,year,unemployment_rate,age_avg,total_population,country,new_building,area_new_b,rooms_new_b,cinema,...,scaled_rooms_new_b,scaled_area_new_b,scaled_cinema,scaled_cinema_seat,scaled_museum,scaled_apartment,scaled_restaurant,score,scaled_unemployment_rate,scaled_employment_rate
0,Baden-Württemberg,2020,4.1,43.240650,11103043,Germany,26600,1402,186843,252.0,...,0.618560,0.852220,0.220814,0.142081,0.784282,0.000000,0.556773,5.029991,0.065789,0.934211
1,Bayern,2020,3.6,43.488237,13140183,Germany,40427,1948,283342,284.0,...,1.000000,1.000000,0.188775,0.189097,0.337564,0.132047,0.710754,5.801950,0.000000,1.000000
2,Berlin,2020,9.7,42.149704,3664088,Germany,3043,156,53307,95.0,...,0.434919,0.289389,0.316358,0.569664,0.268316,0.520919,1.000000,3.688181,0.802632,0.197368
3,Brandenburg,2020,6.2,46.722180,2531071,Germany,8237,131,53611,64.0,...,0.969237,0.351131,0.297385,0.197749,0.974735,0.445386,0.100073,5.856196,0.342105,0.657895
4,Bremen,2020,11.2,43.143783,680130,Germany,778,35,6223,12.0,...,0.000000,0.349140,0.071390,1.000000,0.262221,0.466226,0.643403,3.180624,1.000000,0.000000
5,Hamburg,2020,7.6,41.608811,1852478,Germany,2221,139,31096,33.0,...,0.615183,0.507667,0.076425,0.352711,0.000000,0.394256,0.988801,3.560733,0.526316,0.473684
6,Hessen,2020,5.4,43.518179,6293154,Germany,12237,494,97673,128.0,...,0.513223,0.530957,0.151114,0.131490,0.386893,0.051506,0.645965,3.956006,0.236842,0.763158
7,Mecklenburg-Vorpommern,2020,7.8,46.846078,1610774,Germany,3454,68,24681,79.0,...,0.497267,0.286971,1.000000,0.473557,1.000000,0.804288,0.707851,6.642844,0.552632,0.447368
8,Niedersachsen,2020,5.8,44.270348,8003421,Germany,22024,714,155134,177.0,...,0.824418,0.603004,0.203634,0.098796,0.577593,0.209557,0.287010,4.756201,0.289474,0.710526
9,Nordrhein-Westfalen,2020,7.5,43.759486,17925570,Germany,30352,821,228644,273.0,...,0.290452,0.311077,0.000000,0.188630,0.137301,0.219996,0.308391,2.661432,0.513158,0.486842


In [90]:
columns = ['age_avg', 'employment_rate', 'new_building', 'area_new_b', 'rooms_new_b', 'cinema', 'cinema_seat', 'museum',
       'apartment', 'restaurant']

default_weights = dict(zip(columns, [1] * len(columns)))

def calc_score(weights=default_weights):
    data['w_score'] = [0] * 16
    for col in columns:
        data['w_score'] += data['scaled_%s' % col] * weights[col]
    data.w_score = round(data.w_score,2)
    return data.sort_values(by='w_score', ascending=False).reset_index()[['state','w_score']]

In [68]:
calc_score()

,state,w_score
0,Mecklenburg-Vorpommern,6.642844
1,Brandenburg,5.856196
2,Bayern,5.801950
3,Schleswig-Holstein,5.421472
4,Rheinland-Pfalz,5.127841
5,Baden-Württemberg,5.029991
6,Sachsen,4.943111
7,Niedersachsen,4.756201
8,Thüringen,4.354846
9,Sachsen-Anhalt,4.191425


In [91]:
calc_score({'age_avg':1, 'new_building':2, 
              'area_new_b':2, 'rooms_new_b':2, 
              'cinema':1, 'cinema_seat':0,
              'museum':1,'apartment':2,
              'restaurant':1, 'employment_rate': 2})

,state,w_score
0,Bayern,9.67
1,Brandenburg,9.08
2,Schleswig-Holstein,8.86
3,Mecklenburg-Vorpommern,8.75
4,Rheinland-Pfalz,8.16
5,Baden-Württemberg,7.94
6,Niedersachsen,7.80
7,Sachsen,7.55
8,Thüringen,6.49
9,Hessen,6.14
